# PROGETTO FCS 2020-2021

In [1]:
import numpy as np
import pandas as pd
import math
import re
import datetime as dt
from difflib import get_close_matches

Si importano i datasets.

In [2]:
times = pd.read_csv("timesData.csv")
shangai = pd.read_csv("shanghaiData.csv")
cwur = pd.read_csv("cwurData.csv")

cwur ha il nome della colonna riferita al nome delle università diverso dagli altri ranking. Si procede a normalizzare tale nome.

In [3]:
cwur.rename(columns={"institution": "university_name"}, inplace = True)

shangai ha alcune righe vuote che si eliminano.

In [4]:
shangai = shangai.loc[shangai.university_name.notnull()]

### Normalizzazione nomi università

Poichè nei diversi ranking sono utilizzati nomi diversi per la stessa università, è necessario apportare delle modifiche così da poter poi operare dei confronti tra i diversi ranking per la stessa università. \
Si è inoltre notato che all'interno del ranking 'shangai' alcune università hanno nomi diversi a seconda dell'anno, quindi si decide di utilizzare il ranking *'times'* come riferimento per i nomi, e si assegnano così i nomi di *'times'* ai ranking *'shangai'* e *'cwur'*.

Per prima cosa si portano tutti i nomi in minuscolo.

In [5]:
def normalize_universities(row):
    return row['university_name'].lower()

times['university_name'] = times.apply(normalize_universities, axis = 1)
shangai['university_name'] = shangai.apply(normalize_universities, axis = 1)
cwur['university_name'] = cwur.apply(normalize_universities, axis = 1)

Per prima cosa si ricercano, tra ranking diversi, i nomi diversi ma simili, servendosi della libreria **difflib**, che offre la funzione **get_close_matches** che confronta delle stringhe e, a seconda di un *cutoff* fissato dall'utente, restituisce i match per cui la *normalized edit distance* eccede il *cutoff*. \
Inoltre se il nome presente in *'shangai'* o *'cwur'* è già presente in *'times'* il confronto non è operato. \ 
I match così ottenuti sono poi valutati manualmente per rimuovere le università diverse ma con nomi simili. \
Infine in *'shangai'* e *'cwur'* sono sostituiti i nomi con quelli presenti in *'times'*.

In [6]:
def alternate_names_finder(df1, times = times):
    outputs = {}
    for university1 in set(df1['university_name']):
        norm_university1 = str.replace(university1, 'university', '')
        norm_university1 = str.replace(norm_university1, ' of ', '')
        for university2 in set(times['university_name']):
            if ((university1 != university2) & (university1 not in set(times['university_name']))):
                norm_university2 = str.replace(university2, 'university', '')
                norm_university2 = str.replace(norm_university2, ' of ', '')
                out = get_close_matches(norm_university1, [norm_university2], n = 1, cutoff = 0.80)
                if (len(out) != 0):
                    outputs[university1] = university2
    return outputs

def outputs_cleaner(outputs, no_keys):
    for key in no_keys:
        del outputs[key]
    return outputs

def pulizia_università(row, outputs):
    if (row['university_name'] in list(outputs.keys())):
        return outputs[row['university_name']]
    else:
        return row['university_name']

In [7]:
outputs_shangai = alternate_names_finder(shangai)

In [8]:
outputs_cwur = alternate_names_finder(cwur)

In [9]:
outputs_cwur

{'nanjing university of science and technology': 'jordan university of science and technology',
 'rovira i virgili university': 'university of rovira i virgili',
 'university of western sydney': 'western sydney university',
 'university of castilla–la mancha': 'university of castilla-la mancha',
 'university of milan - bicocca': 'university of milan-bicocca',
 'federal university of paraná': 'federal university of paraná (ufpr)',
 'technion – israel institute of technology': 'technion israel institute of technology',
 'south china normal university': 'east china normal university',
 'pennsylvania state university, university park': 'pennsylvania state university',
 'tokyo medical and dental university': 'tokyo medical and dental university (tmdu)',
 'hefei university of technology': 'delft university of technology',
 'rio de janeiro state university': 'rio de janeiro state university (uerj)',
 "xi'an jiaotong university": 'xi’an jiaotong university',
 'lille 2 university of health and 

Si procede con la ricerca manuale dei match spuri.

In [10]:
no_keys_shangai = ['nara institute of science and technology', 'espci paristech', 'university of kansas medical center', 'university of genova', 'king abdullah university of science and technology', 'curtin university of technology', 'weizmann institute of science', 'federal university of sao paulo', 'capital university of medical sciences', 'royal institute of technology', 'the hong kong university of science and technology', 'university of montpellier 2']
no_keys_cwur = ['ulsan national institute of science and technology', 'école normale supérieure de cachan', 'école normale supérieure - paris', 'grenoble institute of technology', 'federal university of são paulo', 'wrocław university of technology', 'university of north dakota',  'saitama medical university', 'toho university', 'pukyong national university', 'nara institute of science and technology', 'nanjing university of science and technology', 'trent university', 'darmstadt university of technology', 'kyoto institute of technology', 'federal university of santa maria', 'nanjing agricultural university', 'hefei university of technology', 'south china normal university', 'central china normal university', 'king abdullah university of science and technology', 'southern medical university', 'weizmann institute of science', 'royal institute of technology', 'nagoya institute of technology', 'university of salerno', 'national dong hwa university', 'zhejiang university of technology', 'university of montpellier 1', 'university of montpellier 2']


In [11]:
outputs_shangai = outputs_cleaner(outputs_shangai, no_keys_shangai)
outputs_cwur = outputs_cleaner(outputs_cwur, no_keys_cwur)

In [12]:
shangai['university_name'] = shangai.apply(pulizia_università, args = [outputs_shangai], axis = 1)
cwur['university_name'] = cwur.apply(pulizia_università, args = [outputs_cwur], axis = 1)

Si nota la presenza della Università di Montpellier sotto vari pseudonimi.

In [13]:
shangai.loc[shangai.university_name.str.contains('montpellier')]

,world_rank,university_name,national_rank,total_score,alumni,award,hici,ns,pub,pcp,year
277,203-300,university of montpellier 2,9-13,NaN,14.0,0.0,13.6,16.9,31.2,16.7,2005
777,201-300,university of montpellier 2,7-12,NaN,13.5,0.0,13.3,16.6,33.1,18.5,2006
1282,203-304,university of montpellier 2,8-12,NaN,13.2,0.0,12.8,17.3,31.3,17.6,2007
1789,201-302,university of montpellier 2,8-14,NaN,12.5,0.0,12.6,17.0,32.1,41.5,2008
2292,201-302,university of montpellier 2,8-14,NaN,12.2,0.0,12.6,14.4,32.9,17.6,2009
2792,201-300,university of montpellier 2,8-13,NaN,11.9,0.0,12.5,15.6,32.4,17.5,2010
3291,201-300,university of montpellier 2,9-13,NaN,11.8,0.0,12.5,16.6,31.4,17.0,2011
3795,201-300,university of montpellier 2,9-13,NaN,10.8,0.0,12.5,15.5,30.2,16.5,2012
4174,201-300,university of montpellier 2,9-14,NaN,10.5,0.0,6.3,15.0,31.1,17.3,2014
4676,201-300,montpellier university,9-15,NaN,10.3,0.0,6.3,17.7,35.9,14.1,2015


In [14]:
set(shangai.loc[shangai.university_name.str.contains('montpellier')]['university_name'])

{'montpellier university', 'university of montpellier 2'}

In [15]:
set(times.loc[times.university_name.str.contains('montpellier')]['university_name'])

{'montpellier university'}

In [16]:
set(cwur.loc[cwur.university_name.str.contains('montpellier')]['university_name'])

{'montpellier university',
 'paul valéry university, montpellier iii',
 'university of montpellier 1',
 'university of montpellier 2'}

Una rapida ricerca su internet (https://it.wikipedia.org/wiki/Università_di_Montpellier) ci porta a scoprire che l'Università di Montpellier è stata soppressa nel 1970, da essa sono nate tre università (stesso nome con (1,2,3) in fondo). Essendo i ranking riferiti ad anni più recenti, non risulta ben chiaro a cosa si riferisca la generica 'Montpellier University', se ad una delle tre oppure ad una sorta di aggregato.\
Si decide pertanto di eliminare la Montpellier University, evitando così assunzioni ingiustificate.

In [17]:
times = times.loc[~times.university_name.str.match(r'.*montpellier')].reset_index()
shangai = shangai.loc[~shangai.university_name.str.match(r'.*montpellier')].reset_index()
cwur = cwur.loc[~cwur.university_name.str.match(r'.*montpellier')].reset_index()

La normalizzazione dei nomi delle università è ora completata. \
Non ci si aspetta un risultato perfetto dalla procedura eseguita, 
per cui è possibile che alcune università siano ancora presenti con nomi diversi nei vari rankings, tuttavia ci si 
aspetta che ciò avvenga per pochi casi, quindi che l'effetto sia trascurabile.

### Normalizzazione nomi paesi

Si nota che in times sono presenti alcuni nomi di paese errati, come *Unisted States of America* e *Unted Kingdom*. \
Poichè in seguito sarà richiesto di raggruppare per paese è necessario correggere tali occorrenze.\
Si ripete il procedimento già seguito per la normalizzazione dei nomi delle università, come riferimento si prende la lista dei paesi presenti nel dataset *school_and_country_table.csv*.\
Come prima cosa si carica il dataset e si portano tutti i nomi dei paesi in minuscolo.

In [18]:
times.loc[times.country == 'Unted Kingdom']

,index,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
2200,2205,401-500,aston university,Unted Kingdom,18.4,92.5,20.4,51.9,34.6,-,NaN,NaN,NaN,NaN,2016


In [19]:
countries = pd.read_csv("school_and_country_table.csv")
countries.country = countries.apply(lambda row: row['country'].lower(), axis = 1)
times.country = times.apply(lambda row: row['country'].lower(), axis = 1)

In [20]:
def wrong_country_names_finder(times = times, countries = countries):
    outputs = {}
    for country in times.country:
        country = country.lower()
        if country not in set(countries['country']):
            out = get_close_matches(country, list(countries.country), n = 1, cutoff = 0.80)
            if (len(out) != 0):
                outputs[country] = out[0]
    return outputs

In [21]:
outputs = wrong_country_names_finder(times)

In [22]:
outputs

{'unisted states of america': 'united states of america',
 'unted kingdom': 'united kingdom'}

A differenza di prima ci sono solo due nomi di paese errati.

In [23]:
def pulizia_paese(row, outputs):
    if (row['country'] in list(outputs.keys())):
        return outputs[row['country']]
    else:
        return row['country'].lower()

In [24]:
times['country'] = times.apply(pulizia_paese, args = [outputs], axis = 1)

In [25]:
set(times.country)

{'argentina',
 'australia',
 'austria',
 'bangladesh',
 'belarus',
 'belgium',
 'brazil',
 'canada',
 'chile',
 'china',
 'colombia',
 'cyprus',
 'czech republic',
 'denmark',
 'egypt',
 'estonia',
 'finland',
 'france',
 'germany',
 'ghana',
 'greece',
 'hong kong',
 'hungary',
 'iceland',
 'india',
 'indonesia',
 'iran',
 'israel',
 'italy',
 'japan',
 'jordan',
 'kenya',
 'latvia',
 'lebanon',
 'lithuania',
 'luxembourg',
 'macau',
 'malaysia',
 'mexico',
 'morocco',
 'netherlands',
 'new zealand',
 'nigeria',
 'norway',
 'oman',
 'pakistan',
 'poland',
 'portugal',
 'qatar',
 'republic of ireland',
 'romania',
 'russian federation',
 'saudi arabia',
 'serbia',
 'singapore',
 'slovakia',
 'slovenia',
 'south africa',
 'south korea',
 'spain',
 'sweden',
 'switzerland',
 'taiwan',
 'thailand',
 'turkey',
 'uganda',
 'ukraine',
 'united arab emirates',
 'united kingdom',
 'united states of america'}

In [26]:
times.loc[times.university_name == 'aston university']

,index,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_male_ratio,year
550,551,350-400,aston university,united kingdom,16.2,84.5,19.9,11.7,26.8,-,NaN,NaN,NaN,NaN,2012
1751,1755,351-400,aston university,united kingdom,19.4,79.7,21.6,45.1,35.7,-,NaN,NaN,NaN,NaN,2015
2200,2205,401-500,aston university,united kingdom,18.4,92.5,20.4,51.9,34.6,-,NaN,NaN,NaN,NaN,2016


### Normalizzazione ranking delle università *('world_rank')*

Possibilità:

- Lasciare invariati i ranking con valori puntuali e per i ranking che utilizzano i range ex-equo, prendere media/minimo/massimo.
- Utilizzare per tutti i ranking gli stessi range, i.e. i più larghi e prendere le medie per ciascuno.
- Lasciare tutto invariato e quando si hanno range ex-equo restituire la differenza sotto forma di intervalli.

E' stata selezionata la prima possibilità per esclusione, infatti la seconda possibilità porta a rinunciare a dell'informazione mentre la terza possibilità non appare conforme alla richiesta che chiede una differenza e non un range.

### Dati mancanti rappresentati con '-'

Si nota la presenza di valori espressi come *'-'*, non conoscendone il significato si suppone che siano valori mancanti. \
Si ripuliscono le varie colonne di times e si convertono al formato più appropriato.

In [27]:
def str_to_num(stringa):
    if pd.isna(stringa) == False:
        stringa = stringa.replace(",","") # alcune colonne, come num_students, usano la virgola per distinguere le migliaia.
        if stringa == "-":
            stringa = float('nan') # i '-' vengono sostituiti con i NaN
        else:
            stringa = float(stringa)
    return stringa

def perc_to_num(stringa):
    if pd.isna(stringa) == False:
        stringa = stringa.replace("%","") 
        stringa = float(stringa) / 100
    return stringa

def ratio_to_num(stringa):
    if pd.isna(stringa) == False:
        if stringa == "-":
            stringa = float('nan')
        else:
            stringa = re.search("(\d*) : (\d*)", stringa).group(1) # si estrae la percentuale di femmine
            stringa = float(stringa) / 100
    return stringa

In [28]:
def times_df_cleaner(row):
    row['income'] = str_to_num(row['income'])
    row['international'] = str_to_num(row['international'])
    row['total_score'] = str_to_num(row['total_score'])
    row['num_students'] = str_to_num(row['num_students'])
    row['international_students'] = perc_to_num(row['international_students'])
    row['female_male_ratio'] = ratio_to_num(row['female_male_ratio'])
    return row

In [29]:
times = times.apply(times_df_cleaner, axis = 1)

Si modifica il nome della colonna *female_male_ratio* che adesso rappresenta il *female_ratio*.

In [30]:
times.rename(columns = {'female_male_ratio' : 'female_ratio'}, inplace = True)

# 1. For each university, extract from the times dataset the most recent and the least recent data, obtaining two separate dataframes

Si estrae l'anno meno recente tramite la funzione *idxmin()*

In [31]:
times_least_recent = times.iloc[times.groupby('university_name').year.idxmin()] 
times_least_recent.head(5)

,index,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_ratio,year
500,501,301-350,aalborg university,denmark,19.0,75.3,20.0,27.1,36.4,NaN,17422.0,15.9,0.15,0.48,2012
501,502,301-350,aalto university,finland,26.2,49.0,22.2,37.5,61.9,NaN,16099.0,24.2,0.17,0.32,2012
166,166,167,aarhus university,denmark,38.1,33.4,55.6,57.3,61.5,49.9,23895.0,13.6,0.14,0.54,2011
476,476,276-300,aberystwyth university,united kingdom,19.8,63.8,15.5,56.6,35.5,NaN,9252.0,19.2,0.18,0.48,2012
2399,2404,601-800,adam mickiewicz university,poland,20.0,25.7,11.0,15.3,28.7,NaN,40633.0,15.6,0.01,0.71,2016


Si estrae l'anno più recente tramite la funzione *idxmax()*

In [32]:
times_most_recent = times.iloc[times.groupby('university_name').year.idxmax()]
times_most_recent.head(5)

,index,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_ratio,year
1999,2003,201-250,aalborg university,denmark,25.1,71.0,28.4,73.8,43.7,NaN,17422.0,15.9,0.15,0.48,2016
2052,2056,251-300,aalto university,finland,31.1,65.4,32.8,62.1,61.6,NaN,16099.0,24.2,0.17,0.32,2016
1904,1908,=106,aarhus university,denmark,36.9,76.8,50.7,79.8,68.3,57.7,23895.0,13.6,0.14,0.54,2016
2101,2105,301-350,aberystwyth university,united kingdom,21.6,72.2,18.9,67.2,31.3,NaN,9252.0,19.2,0.18,0.48,2016
2399,2404,601-800,adam mickiewicz university,poland,20.0,25.7,11.0,15.3,28.7,NaN,40633.0,15.6,0.01,0.71,2016


# 2. For each university, compute the improvement in income between the least recent and the most recent data points


Income contiene dei valori mancanti, per decidere come trattarli è rilevante conoscere quanto siano frequenti.

In [33]:
print("Il numero di missing values per income è pari a: " + str(round(times.loc[times.income.isnull()].shape[0] / times.shape[0] * 100, 1)) + "%")


Il numero di missing values per income è pari a: 8.4%


I missing in income rappresentano l'8% circa, essendo pochi decidiamo di rimuovere le righe contenenti dei missing.

In [34]:
times_income_nomiss = times.loc[times.income.notnull()].reset_index()
times_income_nomiss['income'] = times_income_nomiss['income'].astype(float)

Si ricavano nuovamente i dataframe con gli anni meno e più recenti a partire dal dataframe senza missing.

In [35]:
times_least_recent_income_nomiss = times_income_nomiss.iloc[times_income_nomiss.groupby('university_name').year.idxmin()]
times_most_recent_income_nomiss = times_income_nomiss.iloc[times_income_nomiss.groupby('university_name').year.idxmax()]

Si uniscono i due dataframe appena ottenuti tramite la funzione *merge()* con metodo *'inner'*.

In [36]:
times_university_income_diff = times_least_recent_income_nomiss[['university_name', 'income', 'year']].merge(
    times_most_recent_income_nomiss[['university_name', 'income', 'year']], on = 'university_name', suffixes = ('_least_recent', '_most_recent'))
times_university_income_diff.head(5)

,university_name,income_least_recent,year_least_recent,income_most_recent,year_most_recent
0,aalborg university,36.4,2012,43.7,2016
1,aalto university,61.9,2012,61.6,2016
2,aarhus university,61.5,2011,68.3,2016
3,aberystwyth university,35.5,2012,31.3,2016
4,adam mickiewicz university,28.7,2016,28.7,2016


Si ricava infine il valore della differenza

In [37]:
times_university_income_diff['diff'] = times_university_income_diff['income_most_recent'] - times_university_income_diff['income_least_recent']
times_university_income_diff.head(5)

,university_name,income_least_recent,year_least_recent,income_most_recent,year_most_recent,diff
0,aalborg university,36.4,2012,43.7,2016,7.3
1,aalto university,61.9,2012,61.6,2016,-0.3
2,aarhus university,61.5,2011,68.3,2016,6.8
3,aberystwyth university,35.5,2012,31.3,2016,-4.2
4,adam mickiewicz university,28.7,2016,28.7,2016,0.0


# 3. Find the university with the largest increase computed in the previous point

A partire dal dataframe appena ottenuto si ricerca la differenza massima tramite la funzione *idxmax()*.

In [38]:
times_university_income_diff.iloc[times_university_income_diff['diff'].idxmax()]

university_name        tu dresden
income_least_recent          31.9
year_least_recent            2012
income_most_recent           99.7
year_most_recent             2016
diff                         67.8
Name: 440, dtype: object

# 4. For each ranking, consider only the most recent data point. For each university, compute the maximum difference between the rankings (e.g. for Aarhus University the value is 122-73=49). Notice that some rankings are expressed as a range



Come spiegato all'inizio la strada che si è scelto per questo punto è quella di considerare i valori medi per i ranking espressi come range.

Si estrae la riga riferita all'anno più recente per ogni università nei ranking *shangai* e *cwur*.

In [39]:
shangai_most_recent = shangai.iloc[shangai.groupby('university_name').year.idxmax()]
cwur_most_recent = cwur.iloc[cwur.groupby('university_name').year.idxmax()]
cwur_most_recent.rename(columns={'world_rank': 'world_rank_cwur', 'year': 'year_cwur'}, inplace = True)

/Users/alessandro/opt/anaconda3/lib/python3.7/site-packages/pandas/core/frame.py:4304: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


Si uniscono i rankings in un unico dataframe.

In [40]:
all_rankings = times_most_recent[['university_name', 'world_rank', 'year']].merge(shangai_most_recent[['university_name', 'world_rank', 'year']], on = 'university_name', suffixes = ('_times', '_shangai'), how = 'outer').merge(cwur_most_recent[['university_name', 'world_rank_cwur', 'year_cwur']], on = 'university_name', how = 'outer')
all_rankings.head(5)

,university_name,world_rank_times,year_times,world_rank_shangai,year_shangai,world_rank_cwur,year_cwur
0,aalborg university,201-250,2016.0,301-400,2015.0,565.0,2015.0
1,aalto university,251-300,2016.0,401-500,2015.0,421.0,2015.0
2,aarhus university,=106,2016.0,73,2015.0,122.0,2015.0
3,aberystwyth university,301-350,2016.0,NaN,NaN,814.0,2015.0
4,adam mickiewicz university,601-800,2016.0,NaN,NaN,NaN,NaN


Prima si sostiuiscono i *NaN* con *9999* per facilitare il riconoscimento. \

Si trasformano poi i ranking di *times* e *shangai* in degli intervalli:
- Se il valore del ranking è puntuale oppure è un ex-equo l'*upper limit* coincicerà con il *lower limit*
- Se il valore del ranking è un range si estraggono l'*upper limit* e il *lower limit*
- Se il valore del ranking è mancante sia l'*upper limit* che il *lower limit* saranno dei *NaN*

cwur ha invece solo ranking puntuali, quindi può essere lasciato invariato.

In [41]:
all_rankings[['world_rank_times', 'world_rank_shangai']] = all_rankings[['world_rank_times', 'world_rank_shangai']].fillna('9999')

def ranger(colonna):
    upper=[]
    lower=[]
    for riga in colonna:
        if riga == '9999':
            upper.append(np.nan)
            lower.append(np.nan)
        elif len(riga)<=4:
            r=str.replace(riga,'=','')
            upper.append(r)
            lower.append(r)
        elif len(riga)>4:
            lower.append(riga[0:3])
            upper.append(riga[4:7])
    return upper, lower

all_rankings['times_upper'] = ranger(all_rankings['world_rank_times'])[0]
all_rankings['times_lower'] = ranger(all_rankings['world_rank_times'])[1]
all_rankings['shangai_upper'] = ranger(all_rankings['world_rank_shangai'])[0]
all_rankings['shangai_lower'] = ranger(all_rankings['world_rank_shangai'])[1]

all_rankings['times_upper'] = all_rankings['times_upper'].astype(float)
all_rankings['times_lower'] = all_rankings['times_lower'].astype(float)
all_rankings['shangai_upper'] = all_rankings['shangai_upper'].astype(float)
all_rankings['shangai_lower'] = all_rankings['shangai_lower'].astype(float)

In [42]:
all_rankings.drop(['world_rank_times', 'world_rank_shangai'], axis = 1, inplace = True)

Si estraggono i valori medi dei ranking dai range.

In [43]:
all_rankings['times_average'] = round((all_rankings['times_upper'] + all_rankings['times_lower'])/2)
all_rankings['shangai_average'] = round((all_rankings['shangai_upper'] + all_rankings['shangai_lower'])/2)

In [44]:
all_rankings

,university_name,year_times,year_shangai,world_rank_cwur,year_cwur,times_upper,times_lower,shangai_upper,shangai_lower,times_average,shangai_average
0,aalborg university,2016.0,2015.0,565.0,2015.0,250.0,201.0,400.0,301.0,226.0,350.0
1,aalto university,2016.0,2015.0,421.0,2015.0,300.0,251.0,500.0,401.0,276.0,450.0
2,aarhus university,2016.0,2015.0,122.0,2015.0,106.0,106.0,73.0,73.0,106.0,73.0
3,aberystwyth university,2016.0,NaN,814.0,2015.0,350.0,301.0,NaN,NaN,326.0,NaN
4,adam mickiewicz university,2016.0,NaN,NaN,NaN,800.0,601.0,NaN,NaN,700.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1310,école centrale de lyon,NaN,NaN,881.0,2015.0,NaN,NaN,NaN,NaN,NaN,NaN
1311,école centrale paris,NaN,NaN,548.0,2015.0,NaN,NaN,NaN,NaN,NaN,NaN
1312,école normale supérieure - paris,NaN,NaN,37.0,2015.0,NaN,NaN,NaN,NaN,NaN,NaN
1313,école normale supérieure de cachan,NaN,NaN,721.0,2015.0,NaN,NaN,NaN,NaN,NaN,NaN


Si ricerca la differenza massima, servendosi della funzione *np.nanmax* che è in grado di trattare i *NaN*, in particolare li ignora se è presente almeno un valore non mancante, mentre restituisce un *NaN* se tutti i valori sono mancanti.

In [45]:
def max_diff_rankings(row):
    diff_times_shangai = abs(row['times_average'] - row['shangai_average'])
    diff_times_cwur = abs(row['times_average'] - row['world_rank_cwur'])
    diff_shangai_cwur = abs(row['shangai_average'] - row['world_rank_cwur'])
    return np.nanmax([diff_times_shangai, diff_times_cwur, diff_shangai_cwur])

all_rankings['max_diff'] = all_rankings.apply(max_diff_rankings, axis = 1)

/Users/alessandro/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:5: RuntimeWarning: All-NaN axis encountered
  """


Il warning ci avverte che la funziona *np.nanmax()* ha incontrato delle righe in cui tutte le differenze sono dei *NaN*, in questo caso la funzione restituisce come massimo un *NaN* che è esattamente il comportamento desiderato.

In [46]:
all_rankings.drop(['year_times', 'year_shangai', 'year_cwur', 'times_upper', 'times_lower', 'shangai_upper', 'shangai_lower'], axis = 1).head(30)

,university_name,world_rank_cwur,times_average,shangai_average,max_diff
0,aalborg university,565.0,226.0,350.0,339.0
1,aalto university,421.0,276.0,450.0,174.0
2,aarhus university,122.0,106.0,73.0,49.0
3,aberystwyth university,814.0,326.0,NaN,488.0
4,adam mickiewicz university,NaN,700.0,NaN,NaN
5,agh university of science and technology,782.0,700.0,NaN,82.0
6,aix-marseille university,206.0,276.0,126.0,150.0
7,ajou university,433.0,700.0,NaN,267.0
8,alexandria university,997.0,700.0,NaN,297.0
9,alexandru ioan cuza university,NaN,700.0,NaN,NaN


# 5. Consider only the most recent data point of the times dataset. Compute the number of male and female students for each country.



Si estraggono i numeri di maschi e femmine in ogni università, servendosi dei due attributi *num_students* e *female_ratio*. \
Si osserva che entrambi gli attributi presentano dei valori mancanti.\
Si decide di escludere le università per cui non è noto il numero di studenti, poichè questo è altamente variabile tra le università di uno stesso paese.
Diversamente si decide, per il *female_ratio*, di attribuire ai valori mancanti il valore medio del paese.

In [108]:
mean_std = times_most_recent.loc[times_most_recent.num_students.notnull()].groupby('country').agg({'num_students' : ['mean', 'std'], 'female_ratio' : ['mean', 'std']}).head(15)

In [111]:
mean_std['num_students']['mean']

country
argentina         108373.000000
australia          23987.967742
austria            19211.000000
bangladesh         62716.000000
belarus            29303.000000
belgium            24237.285714
brazil             31452.235294
canada             28682.160000
chile              19337.666667
china              34746.459459
colombia           27322.500000
cyprus              6880.000000
czech republic     22591.222222
denmark            19892.500000
egypt             125396.333333
Name: mean, dtype: float64

In [114]:
mean_std['num_stud_ratio'] = mean_std['num_students']['std'] / mean_std['num_students']['mean']
mean_std['female_ratio_ratio'] = mean_std['female_ratio']['std'] / mean_std['female_ratio']['mean']

mean_std

num_students                female_ratio            \
                         mean            std         mean       std   
country                                                               
argentina       108373.000000            NaN     0.620000       NaN   
australia        23987.967742   10759.248532     0.557333  0.055733   
austria          19211.000000    9733.781451     0.475714  0.173480   
bangladesh       62716.000000            NaN     0.340000       NaN   
belarus          29303.000000            NaN     0.690000       NaN   
belgium          24237.285714   11559.818680     0.548571  0.012150   
brazil           31452.235294   19370.534064     0.482000  0.133426   
canada           28682.160000   12483.292497     0.553913  0.039236   
chile            19337.666667    7277.932280     0.404000  0.087086   
china            34746.459459   13523.263498     0.426071  0.115262   
colombia         27322.500000   16333.459539     0.470000  0.042426   
cyprus            6880.000000            NaN     0.690000       NaN   
czech republic   22591.222222   14803.921928     0.538571  0.119523   
denmark          19892.500000    6110.202051     0.493333  0.129872   
egypt           125396.333333  107576.432109     0.526667  0.106927   

               num_stud_ratio female_ratio_ratio  
                                                  
country                                           
argentina                 NaN                NaN  
australia            0.448527           0.100000  
austria              0.506677           0.364672  
bangladesh                NaN                NaN  
belarus                   NaN                NaN  
belgium              0.476944           0.022148  
brazil               0.615871           0.276817  
canada               0.435228           0.070835  
chile                0.376360           0.215560  
china                0.389198           0.270522  
colombia             0.597803           0.090269  
cyprus                    NaN                NaN  
czech republic       0.655295           0.221926  
denmark              0.307161           0.263254  
egypt                0.857891           0.203026

Oltre a calcolare la deviazione standard si decide di effettuare un confronto tra la media e la sd al fine di depurare il dato dall'ordine di grandezza. Osservando quindi il nuovo parametro *ratio* si nota come questo sia alto nel caso di *num_students*, per questo si ritiene inappropriato sostituire la media per *country* di *num_students* in tutti quei casi in cui tale variabile presenta un valore mancante. Ciò non si verifica per la variabile *female_ratio*, quindi si decide di sostituire la media del paese di appartenenza se l'università presenta un valore nullo per tale variabile.

In [49]:
times_female_no_miss = times_most_recent.loc[times_most_recent.female_ratio.notnull()]

In [50]:
female_means = times_female_no_miss.groupby('country').female_ratio.mean()
female_means

country
argentina                   0.620000
australia                   0.557333
austria                     0.475714
bangladesh                  0.340000
belarus                     0.690000
                              ...   
uganda                      0.500000
ukraine                     0.490000
united arab emirates        0.620000
united kingdom              0.536400
united states of america    0.503357
Name: female_ratio, Length: 70, dtype: float64

In [51]:
def female_mean_extender(row):
    if np.isnan(row['female_ratio']):
        row['female_ratio'] = female_means[row['country']]
    return row

In [52]:
times_most_recent = times_most_recent.apply(female_mean_extender, axis = 1)

Si controlla se effettivamente non sono più presenti università con valore mancante per il *female_ratio*

In [53]:
times_most_recent.loc[times_most_recent.female_ratio.isnull()]

,index,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_ratio,year


Questo ci suggerisce anche che non erano presenti paesi con tutte le università con un valore nullo per *female_ratio*

Si procede ora a calcolare il numero di studenti maschi e femmine per ogni università

In [54]:
times_most_recent.is_copy = False ### per evitare warnings
times_most_recent['female_num'] = round(times_most_recent['num_students'] * (times_most_recent['female_ratio']))
times_most_recent['male_num'] = times_most_recent['num_students'] - times_most_recent['female_num']

Si procede ora ad aggregare per ogni paese.

In [55]:
fem_mal_country=times_most_recent.groupby('country')[['female_num', 'male_num']].sum().reset_index()
fem_mal_country

/Users/alessandro/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  """Entry point for launching an IPython kernel.


,country,female_num,male_num
0,argentina,67191.0,41182.0
1,australia,408596.0,335031.0
2,austria,68364.0,66113.0
3,bangladesh,21323.0,41393.0
4,belarus,20219.0,9084.0
...,...,...,...
65,uganda,18670.0,18670.0
66,ukraine,17846.0,19250.0
67,united arab emirates,9516.0,4931.0
68,united kingdom,711815.0,613027.0


# 6. Find the universities where the ratio between female and male is below the average ratio (computed over all universities)



Si calcola il valore del rapporto *female_ratio* / *male_ratio*. \
Si nota la presenza di una università con sole femmine, per la quale il rapporto *female_ratio* / *male_ratio* vale infinito, si decide di non considerare questa università nel computo del rapporto medio di *female_ratio* / *male_ratio*.

In [56]:
times_most_recent['fm_ratio']=((times_most_recent['female_ratio'])/(1 - times_most_recent['female_ratio']))

Se ne calcola il valore medio, escludendo l'università in cui non sono presenti maschi.

In [57]:
meanfmratio=times_most_recent.loc[times_most_recent['male_num']!=0]['fm_ratio'].mean()
meanfmratio

1.067949202064973

Si estraggono le università dove il valore del *fm_ratio* è sotto la media.

In [58]:
uni_under=times_most_recent.loc[times_most_recent['fm_ratio']<meanfmratio].reset_index()
uni_under

,level_0,index,world_rank,university_name,country,teaching,international,research,citations,income,total_score,num_students,student_staff_ratio,international_students,female_ratio,year,female_num,male_num,fm_ratio
0,1999,2003,201-250,aalborg university,denmark,25.1,71.0,28.4,73.8,43.7,NaN,17422.0,15.9,0.15,0.48,2016,8363.0,9059.0,0.923077
1,2052,2056,251-300,aalto university,finland,31.1,65.4,32.8,62.1,61.6,NaN,16099.0,24.2,0.17,0.32,2016,5152.0,10947.0,0.470588
2,2101,2105,301-350,aberystwyth university,united kingdom,21.6,72.2,18.9,67.2,31.3,NaN,9252.0,19.2,0.18,0.48,2016,4441.0,4811.0,0.923077
3,2401,2406,601-800,ajou university,south korea,19.5,20.0,11.9,23.9,45.7,NaN,12706.0,11.3,0.02,0.33,2016,4193.0,8513.0,0.492537
4,2403,2408,601-800,alexandria university,egypt,20.3,33.7,8.2,14.1,29.7,NaN,127431.0,23.3,0.01,0.46,2016,58618.0,68813.0,0.851852
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
395,2100,2104,251-300,zhejiang university,china,44.3,19.6,46.0,36.1,96.2,NaN,47508.0,15.9,0.05,0.41,2016,19478.0,28030.0,0.694915
396,1852,1856,54,école normale supérieure,france,70.6,85.5,47.7,87.1,37.1,69.0,2400.0,7.9,0.20,0.46,2016,1104.0,1296.0,0.851852
397,2009,2013,201-250,école normale supérieure de lyon,france,41.6,65.6,30.0,69.0,31.7,NaN,2218.0,8.0,0.14,0.49,2016,1087.0,1131.0,0.960784
398,1900,1904,=101,école polytechnique,france,53.5,92.8,44.6,64.7,82.3,57.9,2429.0,4.8,0.30,0.18,2016,437.0,1992.0,0.219512


# 7. For each country compute the fraction of students that are in one of the universities computed in the previous point.



Si prendono i due dataframe contenenti rispettivamente tutte le università e le sole università con rapporto *female_ratio* / *male_ratio* sotto la media, per ciascuna si raggruppa per paese e si ricava il numero di studenti, si calcola infine la frazione per ogni paese.

Si lavora sul dataframe con tutte le università.

In [59]:
ratio_country_tot = times_most_recent.groupby('country')['num_students'].sum().reset_index()
ratio_country_tot

,country,num_students
0,argentina,108373.0
1,australia,743627.0
2,austria,134477.0
3,bangladesh,62716.0
4,belarus,29303.0
...,...,...
65,uganda,37340.0
66,ukraine,37096.0
67,united arab emirates,14447.0
68,united kingdom,1324842.0


Si lavora sul dataframe con le università sotto la media.



In [60]:
ratio_country_under = uni_under.groupby('country')['num_students'].sum().reset_index()
ratio_country_under

,country,num_students
0,australia,146235.0
1,austria,61033.0
2,bangladesh,62716.0
3,brazil,319211.0
4,canada,86779.0
5,chile,116026.0
6,china,1155762.0
7,colombia,54645.0
8,czech republic,38685.0
9,denmark,45705.0


Si mette tutto insieme.

In [61]:
ratio_all = ratio_country_tot.merge(ratio_country_under, on = 'country', suffixes = ('_tot', '_under'))
ratio_all

,country,num_students_tot,num_students_under
0,australia,743627.0,146235.0
1,austria,134477.0,61033.0
2,bangladesh,62716.0,62716.0
3,brazil,534688.0,319211.0
4,canada,717054.0,86779.0
5,chile,116026.0,116026.0
6,china,1285619.0,1155762.0
7,colombia,54645.0,54645.0
8,czech republic,203321.0,38685.0
9,denmark,119355.0,45705.0


Per ogni paese si calcola il rapporto.

In [62]:
ratio_all['fraction']=ratio_all['num_students_under']/ratio_all['num_students_tot']
ratio_all

,country,num_students_tot,num_students_under,fraction
0,australia,743627.0,146235.0,0.196651
1,austria,134477.0,61033.0,0.453855
2,bangladesh,62716.0,62716.0,1.000000
3,brazil,534688.0,319211.0,0.597004
4,canada,717054.0,86779.0,0.121022
5,chile,116026.0,116026.0,1.000000
6,china,1285619.0,1155762.0,0.898993
7,colombia,54645.0,54645.0,1.000000
8,czech republic,203321.0,38685.0,0.190266
9,denmark,119355.0,45705.0,0.382933


# 8. Read the file educational_attainment_supplementary_data.csv, discarding any row without country_name or series_name



In [63]:
eduatt = pd.read_csv("educational_attainment_supplementary_data.csv")
eduatt

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,0.57,...,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN,NaN
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,1.54,...,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.83,NaN,NaN,0.95,NaN,NaN,NaN,1.26,...,1.01,NaN,NaN,NaN,NaN,2.45,NaN,NaN,NaN,NaN
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2.34,NaN,NaN,2.22,NaN,NaN,NaN,2.37,...,2.26,NaN,NaN,NaN,NaN,3.55,NaN,NaN,NaN,NaN
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.54,NaN,NaN,0.92,NaN,NaN,NaN,0.94,...,2.00,NaN,NaN,NaN,NaN,1.29,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79050,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79051,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79052,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79053,Data from database: Education Statistics: Educ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [64]:
eduatt=eduatt[eduatt['country_name'].notnull() & eduatt['series_name'].notnull()]
eduatt

,country_name,series_name,1985,1986,1987,1990,1991,1992,1993,1995,...,2005,2006,2007,2008,2009,2010,2011,2012,2013,2015
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.33,NaN,NaN,0.44,NaN,NaN,NaN,0.57,...,0.86,NaN,NaN,NaN,NaN,1.27,NaN,NaN,NaN,NaN
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1.03,NaN,NaN,1.26,NaN,NaN,NaN,1.54,...,2.18,NaN,NaN,NaN,NaN,2.64,NaN,NaN,NaN,NaN
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.83,NaN,NaN,0.95,NaN,NaN,NaN,1.26,...,1.01,NaN,NaN,NaN,NaN,2.45,NaN,NaN,NaN,NaN
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",2.34,NaN,NaN,2.22,NaN,NaN,NaN,2.37,...,2.26,NaN,NaN,NaN,NaN,3.55,NaN,NaN,NaN,NaN
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",0.54,NaN,NaN,0.92,NaN,NaN,NaN,0.94,...,2.00,NaN,NaN,NaN,NaN,1.29,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79045,Zimbabwe,UIS: Percentage of population age 25+ with som...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.84323,NaN,NaN
79046,Zimbabwe,UIS: Percentage of population age 25+ with som...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,16.68491,NaN,NaN
79047,Zimbabwe,UIS: Percentage of population age 25+ with unk...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.99412,NaN,NaN
79048,Zimbabwe,UIS: Percentage of population age 25+ with unk...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5.77150,NaN,NaN


# 9. From attainment build a dataframe with the same data, but with 4 columns: country_name, series_name, year, value



In [65]:
new_eduatt = pd.melt(eduatt, id_vars=['country_name', 'series_name'], var_name='year')
new_eduatt

,country_name,series_name,year,value
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.33
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,1.03
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.83
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,2.34
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.54
...,...,...,...,...
2134345,Zimbabwe,UIS: Percentage of population age 25+ with som...,2015,NaN
2134346,Zimbabwe,UIS: Percentage of population age 25+ with som...,2015,NaN
2134347,Zimbabwe,UIS: Percentage of population age 25+ with unk...,2015,NaN
2134348,Zimbabwe,UIS: Percentage of population age 25+ with unk...,2015,NaN


Le colonne che non sono specificate in *id_vars* vengono trasformate in due nuove variabili *year* e *value*. 

In [66]:
new_eduatt = new_eduatt.loc[new_eduatt['value'].notnull()]
new_eduatt

,country_name,series_name,year,value
0,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.33000
1,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,1.03000
2,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.83000
3,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,2.34000
4,Afghanistan,"Barro-Lee: Average years of primary schooling,...",1985,0.54000
...,...,...,...,...
2054020,West Bank and Gaza,UIS: Percentage of population age 25+ with som...,2013,1.48356
2054021,West Bank and Gaza,UIS: Percentage of population age 25+ with som...,2013,1.88820
2054022,West Bank and Gaza,UIS: Percentage of population age 25+ with unk...,2013,0.00000
2054023,West Bank and Gaza,UIS: Percentage of population age 25+ with unk...,2013,0.00000


# 10. For each university, find the number of rankings in which they appear (it suffices to appear in one year for each ranking).



In [68]:
def count_generator(df, name_df):
    return pd.DataFrame(list(zip(set(df['university_name']), np.repeat(1, len(set(df['university_name'])), axis = 0))),
                        columns = ['university_name', name_df])

La funzione attesta la presenza di un'università all'interno di ogni ranking.

In [69]:
times_unis = count_generator(df = times, name_df = 'times')
shangai_unis = count_generator(df = shangai, name_df = 'shangai')
cwur_unis = count_generator(df = cwur, name_df = 'cwur')

In [70]:
times_unis

,university_name,times
0,university of greifswald,1
1,shahid beheshti university,1
2,university of notre dame,1
3,ewha womans university,1
4,jilin university,1
...,...,...
812,pontifical catholic university of rio de janei...,1
813,prince of songkla university,1
814,osaka city university,1
815,oregon state university,1


In [71]:
all_unis = times_unis.merge(
    shangai_unis, on = 'university_name', how = 'outer').merge(
        cwur_unis,  on = 'university_name', how = 'outer')

In [72]:
all_unis.fillna(0, inplace = True)
all_unis['times'] = all_unis['times'].astype(int)
all_unis['shangai'] = all_unis['shangai'].astype(int)
all_unis['cwur'] = all_unis['cwur'].astype(int)

Si sostituiscono i valori mancanti con uno 0 che sta ad indicare l'assenza dell'università in questione nel ranking considerato. Facendo poi una semplice somma si ottiene il valore corrispondente al numero di ranking in cui figura l'università considerata.

In [73]:
all_unis['presence'] = all_unis['times'] + all_unis['shangai'] + all_unis['cwur']

In [74]:
all_unis

,university_name,times,shangai,cwur,presence
0,university of greifswald,1,1,1,3
1,shahid beheshti university,1,0,0,1
2,university of notre dame,1,1,1,3
3,ewha womans university,1,1,1,3
4,jilin university,1,1,1,3
...,...,...,...,...,...
1310,jichi medical university,0,0,1,1
1311,georgia regents university,0,0,1,1
1312,inje university,0,0,1,1
1313,east carolina university,0,0,1,1


# 11. In the times ranking, compute the number of times each university appears



In [75]:
years_count = times.groupby('university_name').year.count().reset_index()
years_count.rename(columns={"year": "presences"}, inplace = True)

university_name
aalborg university                          5
aalto university                            5
aarhus university                           6
aberystwyth university                      5
adam mickiewicz university                  1
                                           ..
école normale supérieure                    6
école normale supérieure de lyon            6
école polytechnique                         6
école polytechnique fédérale de lausanne    6
örebro university                           1
Name: year, Length: 817, dtype: int64

In [ ]:
years_count

# 12. Find the universities that appear at most twice in the times ranking.



In [78]:
years_count.loc[years_count.presences <= 2]

,university_name,presences
4,adam mickiewicz university,1
5,agh university of science and technology,1
6,aix-marseille university,1
7,ajou university,1
9,alexandru ioan cuza university,1
...,...,...
805,yokohama city university,1
806,yokohama national university,1
809,yuan ze university,2
810,yıldız technical university,1


# 13. The universities that, in any year, have the same position in all three rankings (they must have the same position in a year).

Si esegue un merge sulla base del nome dell'università e dell'anno.

In [79]:
total = times[['university_name', 'year', 'world_rank']].merge(
    shangai[['university_name', 'year', 'world_rank']], on = ['university_name', 'year'], how = 'inner').merge(
    cwur[['university_name', 'year', 'world_rank']],  on = ['university_name', 'year'], how = 'inner')

In [80]:
total.rename(columns = {'world_rank_x' : 'times', 'world_rank_y': 'shangai', 'world_rank': 'cwur'}, inplace = True)

In [81]:
total

,university_name,year,times,shangai,cwur
0,california institute of technology,2012,1,6,5
1,harvard university,2012,2,1,1
2,stanford university,2012,2,2,3
3,university of oxford,2012,4,10,7
4,princeton university,2012,5,7,6
...,...,...,...,...,...
694,university of turku,2015,351-400,301-400,383
695,university of vermont,2015,351-400,301-400,324
696,vrije universiteit brussel,2015,351-400,201-300,280
697,washington state university,2015,351-400,301-400,31


In *Times* e *Shangai* oltre una certa posizione in classifica non viene più riportato il valore puntuale ma un range. Si estraggono quindi gli estremi di tale range.

In [82]:
total['times_lower'] = ranger(total['times'])[1]
total['times_upper'] = ranger(total['times'])[0]
total['shangai_lower'] = ranger(total['shangai'])[1]
total['shangai_upper'] = ranger(total['shangai'])[0]


total['times_upper'] = total['times_upper'].astype(float)
total['times_lower'] = total['times_lower'].astype(float)
total['shangai_upper'] = total['shangai_upper'].astype(float)
total['shangai_lower'] = total['shangai_lower'].astype(float)

In [83]:
total

,university_name,year,times,shangai,cwur,times_lower,times_upper,shangai_lower,shangai_upper
0,california institute of technology,2012,1,6,5,1.0,1.0,6.0,6.0
1,harvard university,2012,2,1,1,2.0,2.0,1.0,1.0
2,stanford university,2012,2,2,3,2.0,2.0,2.0,2.0
3,university of oxford,2012,4,10,7,4.0,4.0,10.0,10.0
4,princeton university,2012,5,7,6,5.0,5.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...
694,university of turku,2015,351-400,301-400,383,351.0,400.0,301.0,400.0
695,university of vermont,2015,351-400,301-400,324,351.0,400.0,301.0,400.0
696,vrije universiteit brussel,2015,351-400,201-300,280,351.0,400.0,201.0,300.0
697,washington state university,2015,351-400,301-400,31,351.0,400.0,301.0,400.0


Le posizioni si considerano come corrispondenti se il valore puntuale presente in *cwur* ricade all'interno dell'intervallo riportato in *times* e *shangai*, ciò assicura anche che i due intervalli si tocchino, avendo essi sicuramente almeno un punto in comune. 

In [84]:
def position_match(row):
    condition1 = (row.cwur <= row.times_upper) & (row.cwur >= row.times_lower)
    condition2 = (row.cwur <= row.shangai_upper) & (row.cwur >= row.shangai_lower) 
    if (condition1 & condition2):
        return 1
    else:
        return 0

total['match'] = total.apply(position_match, axis = 1)




In [86]:
match=total.drop(['times', 'shangai'], axis = 1).groupby('university_name').match.sum().reset_index().sort_values('match', ascending = False)

,university_name,match
63,linköping university,2
238,university of southampton,1
257,university of turku,1
126,temple university,1
84,newcastle university,1
...,...,...
101,queen’s university belfast,0
100,queen’s university,0
99,queensland university of technology,0
98,queen mary university of london,0


In [ ]:
match.loc[match.match>0]

## Bicocca

In [87]:
stotal = times[['university_name', 'year', 'world_rank']].merge(
    shangai[['university_name', 'year', 'world_rank']], on = ['university_name', 'year'], how = 'outer').merge(
    cwur[['university_name', 'year', 'world_rank']],  on = ['university_name', 'year'], how = 'outer')

stotal.loc[stotal.university_name.str.match(r'.*bicocca')].sort_values('year')

,university_name,year,world_rank_x,world_rank_y,world_rank
5914,university of milan-bicocca,2011,NaN,401-500,NaN
445,university of milan-bicocca,2012,226-250,NaN,NaN
870,university of milan-bicocca,2013,251-275,NaN,NaN
1242,university of milan-bicocca,2014,226-250,301-400,431.0
1646,university of milan-bicocca,2015,226-250,401-500,382.0
2124,university of milan-bicocca,2016,301-350,NaN,NaN
